<a href="https://colab.research.google.com/github/samitha278/gpt-from-scratch/blob/main/GPT_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import gdown
import random

In [10]:
file_id = "1ia6z4itw7WJWpnoTohURX6Lm-AnZmVZz"
url = f"https://drive.google.com/uc?id={file_id}"

output = "input.txt"
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1ia6z4itw7WJWpnoTohURX6Lm-AnZmVZz
To: /content/input.txt
100%|██████████| 1.12M/1.12M [00:00<00:00, 10.1MB/s]


'input.txt'

In [11]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

len(text)

1115394

In [12]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [13]:
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for i,s in enumerate(chars)}

encode = lambda s : [stoi[ch] for ch in s]
decode = lambda l : ''.join(itos[i] for i in l)

print(encode("Hello world"))
print(decode(encode("Hello world")))

[20, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
Hello world


In [14]:
data = torch.tensor(encode(text))
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [16]:
n = int(0.9* len(data))
train = data[:n]
val = data[n:]

In [17]:
block_size = 8

x = train[:block_size]
y = train[1:block_size+1]

for i in range(block_size):
  context = x[:i+1]
  target = y[i]

  print(f'{context} target:{target}')

tensor([18]) target:47
tensor([18, 47]) target:56
tensor([18, 47, 56]) target:57
tensor([18, 47, 56, 57]) target:58
tensor([18, 47, 56, 57, 58]) target:1
tensor([18, 47, 56, 57, 58,  1]) target:15
tensor([18, 47, 56, 57, 58,  1, 15]) target:47
tensor([18, 47, 56, 57, 58,  1, 15, 47]) target:58
